In [1]:
from data import RooftopDataset
from transforms import ToTensor
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch
from torch import optim
import numpy as np
from engine import evaluate
import utils

In [2]:
#### REDRAW PICTURES WHEREBY EACH ROOF GETS ITS OWN COLOR #####
## ONLY NEEDS TO BE DONE ONCE ##
# data_root_path = 'images'
# dataset = RooftopDataset(root=data_root_path, label_subfolder='labels',
                         #grayscale=True
#                         )
# dataset.save_color_encoded_images()

In [3]:
dataset = RooftopDataset(root='images',
                         #grayscale=True,
                         transforms=ToTensor())

In [4]:
# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, min_size=256)

# replace the classifier with a new one
num_classes = 2  # 1 class (rooftop) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [5]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True, min_size=128)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [6]:
def train_one_epoch(model, optimizer, data_loader):
    model.train()

    for i, (images, targets) in enumerate(data_loader):
        targets = list(target for target in targets)
        
        loss_dict = model(images, targets)
        print(loss_dict)

        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

In [7]:
# get the model using our helper function
model = get_model_instance_segmentation(num_classes)
# let's train it for 10 epochs
num_epochs = 10
optimizer = optim.Adam
for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader)
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

print("That's it!")

NameError: name 'data_loader' is not defined

In [8]:
def main():
    # train on the GPU or on the CPU, if a GPU is not available
    # device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    device = torch.device('cpu')
    # our dataset has two classes only - background and person
    num_classes = 2
    
    to_grayscale = True

    dataset = RooftopDataset(root='images',
                             transforms=ToTensor(),
                             #grayscale=to_grayscale
                            )
    # split the dataset in train and test set
    indices = torch.randperm(len(dataset)).tolist()
    dataset_test = RooftopDataset(root='images', transforms=ToTensor())

    dataset = torch.utils.data.Subset(dataset, indices[:-2])
    dataset_test = torch.utils.data.Subset(dataset_test, indices[-2:])

    data_loader = torch.utils.data.DataLoader(
            dataset, batch_size=2, shuffle=True, num_workers=1,
            collate_fn=utils.collate_fn
    )

    data_loader_test = torch.utils.data.DataLoader(
            dataset_test, batch_size=1, shuffle=False, num_workers=1,
            collate_fn=utils.collate_fn
    )

    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    # model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)
    #optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # let's train it for 10 epochs
    num_epochs = 10

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader)
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

    print("That's it!")

In [9]:
%tb

main()

NameError: name 'data_loader' is not defined

{'loss_classifier': tensor(1.0671, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(0.2590, grad_fn=<DivBackward0>), 'loss_mask': tensor(2.0570, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(0.5335, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(0.0451, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(0.2555, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(0.1113, grad_fn=<DivBackward0>), 'loss_mask': tensor(1.6255, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(0.2298, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(0.0226, grad_fn=<DivBackward0>)}
{'loss_classifier': tensor(0.2759, grad_fn=<NllLossBackward>), 'loss_box_reg': tensor(0.2417, grad_fn=<DivBackward0>), 'loss_mask': tensor(1.0179, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_objectness': tensor(0.3915, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), 'loss_rpn_box_reg': tensor(0.0325, grad_fn=

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

In [13]:
import os
import numpy as np
from PIL import Image
import torch
from scipy import ndimage


class RooftopDataset(object):
    def __init__(self, root, grayscale=False, transforms=None, label_subfolder="color_encoded"):
        self.root = root
        self.grayscale = grayscale
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "images"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, label_subfolder))))
        self.label_subfolder = label_subfolder

    def __getitem__(self, idx):
        # load images and masks

        img_path = os.path.join(self.root, "images", self.imgs[idx])
        mask_path = os.path.join(self.root, self.label_subfolder, self.masks[idx])
        with Image.open(img_path) as img:
            if self.grayscale:
                img = img.convert("L")
            else:
                img = img.convert("RGB")
        mask = Image.open(mask_path)
        mask_array = np.array(mask)

        boxes = []
        # find different objects (roofs) in mask
        label_img, nr_labels = ndimage.label(mask_array)
        print(f'uniques: {np.unique(mask_array)}')
        print(f'nr labels {nr_labels}')
        for i in range(1, nr_labels + 1):
            pos = np.where(label_img == i)
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])
        labels = torch.ones((nr_labels,), dtype=torch.int64)

        # instances are encoded as different colors
        obj_ids = np.unique(label_img)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]
        masks = label_img == obj_ids[:, None, None]

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((nr_labels,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        # suppose all instances are not crowd (artifact from coco pretrained net)
        iscrowd = torch.zeros((nr_labels,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

In [24]:
class RooftopDataset(object):
    def __init__(self, root, grayscale=False, transforms=None, label_subfolder="color_encoded"):
        self.root = root
        self.grayscale = grayscale
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "images"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, label_subfolder))))

    def __getitem__(self, idx):
        # load images and masks

        img_path = os.path.join(self.root, "images", self.imgs[idx])
        mask_path = os.path.join(self.root, "labels", self.masks[idx])
        with Image.open(img_path) as img:
            if self.grayscale:
                img = img.convert("L")
            else:
                img = img.convert("RGB")
        mask = Image.open(mask_path)
        mask = np.array(mask)
        for i in mask:
            for x in i:
                if x!=0:
                    print(x)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        print(f'number objs: {len(obj_ids)}')
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]
        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        # suppose all instances are not crowd (artifact from coco pretrained net)
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target


In [25]:
indices = torch.randperm(len(dataset)).tolist()
dataset_test = RooftopDataset(root='images', transforms=ToTensor())
dataset = torch.utils.data.Subset(dataset, indices[:-2])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-2:])

data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=1,
        collate_fn=utils.collate_fn
)

data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=1,
        collate_fn=utils.collate_fn
)

In [26]:
data = data_loader_test.__iter__().next()

1
25
12
1
1
2
8
30
11
23
24
68
49
4
1
5
14
25
33
44
132
239
164
33
1
1
1
9
29
26
176
251
255
232
57
24
2
31
21
80
143
167
212
255
255
251
255
180
37
1
19
40
104
182
218
241
255
255
255
255
253
254
255
242
92
33
1
1
8
37
68
184
247
255
255
255
255
254
254
254
255
255
253
255
196
27
7
1
5
31
17
162
236
255
255
253
253
254
254
254
255
255
255
255
253
255
249
119
35
1
29
29
135
223
255
255
253
253
255
255
255
255
255
255
255
255
255
255
252
255
209
17
18
39
96
201
254
255
255
253
255
255
255
255
255
255
255
255
255
255
255
255
254
255
255
140
33
1
8
36
58
180
245
255
255
253
254
255
255
255
255
255
255
255
255
255
255
255
255
255
255
253
255
222
17
1
6
31
14
158
233
255
255
253
254
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
254
255
161
39
2
1
29
28
128
220
255
255
253
253
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
253
255
233
57
27
1
17
39
97
201
254
255
255
253
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255

253
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
255
255
155
31
1
28
35
121
216
255
255
255
253
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
252
255
231
41
25
17
40
98
197
252
255
255
253
254
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
255
163
30
2
1
17
33
55
181
245
255
255
253
254
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
253
255
226
17
1
7
34
161
233
255
255
253
254
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255


255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
253
255
255
236
162
39
21
1
1
9
36
56
181
245
255
255
252
254
255
255
255
255
255
255
255
255
255
253
255
217
25
1
21
219
255
253
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
253
255
255
245
180
69
40
17
1
1
34
23
156
234
255
255
253
254
255
255
255
255
255
255
255
255
255
255
255
254
254
255
183
26
9
35
150
255
255
254
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
252
255
255
250
196
98
34
27
1
21
37
123
216
255
255
254
253
255
255
255
255
255
255
255
255
255
255
255
255
255
255
253
255
255
141
38
1
1
27
47
230
255
253
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
2

255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
253
255
212
15
7
38
159
255
255
254
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
253
255
244
99
33
1
1
12
25
191
255
254
254
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
253
253
255
255
144
38
1
1
27
221
255
253
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
253
255
255
231
151
31
7
1
37
108
247
255
253
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
253
255
255
246
177
42
12
11
23
196
255
253
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255


255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
255
249
109
30
1
35
101
245
255
253
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
255
255
144
36
1
27
41
230
255
253
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
254
255
175
37
1
12
9
205
255
253
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
253
255
200
27
1
1
38
172
255
255
254
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
253
255
221
14
1
40
136
254
255
254
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
2

255
255
254
253
254
253
253
253
253
253
253
253
254
254
254
254
254
254
254
254
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
253
255
239
77
32
1
15
223
255
253
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
255
245
91
26
1
32
76
237
255
251
253
254
254
254
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
253
255
250
118
36
11
10
205
255
253
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
255
252
123
33
23
221
255
253
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
255


254
252
255
226
5
16
1
27
63
235
255
252
254
255
255
255
255
255
255
255
255
255
247
235
224
208
189
165
142
118
84
37
29
36
37
34
27
20
14
10
5
2
28
86
243
255
253
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
255
254
254
254
254
253
253
253
253
253
253
253
253
254
253
253
254
255
255
255
255
255
255
255
255
255
255
255
231
17
16
228
255
255
255
255
252
243
232
215
200
181
154
132
104
65
3
21
36
40
38
32
23
11
25
47
230
255
253
255
255
255
254
254
254
254
253
253
253
252
252
253
253
253
253
253
254
254
254
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
254
255
248
242
244
240
234
200
2
13
2
16
182
217
193
172
145
123
87
43
18
30
35
35
33
27
19
7
1
1
1
1
24
6
224
255
251
254
253
254
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
254
255
248
241
243
241
227
224
227
216
201
202
202
180
170
175
164
135
134
138
111
84
91
84
18
5
1
17
26
29
29
29
26
24
19
11
2
1
1
1
20
218
255
255
255
255
255
255
255

255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
253
255
255
255
245
207
143
30
16
30
23
6
1
6
9
12
15
19
25
28
34
35
37
34
29
16
51
87
119
138
160
181
199
213
233
229
84
27
1
33
125
252
255
254
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
253
254
255
255
255
229
177
100
36
27
8
1
14
26
18
3
43
84
104
133
147
170
185
204
219
231
242
250
255
255
255
255
255
255
255
119
30
13
218
255
253
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
253
253
255
255
255
246
207
141
39
7
40
22
15
43
83
102
131
145
169
181
201
209
225
231
242
246
254
255
255
255
255
255
255
255
255
255
255
254
253
252
253
254
133
30
3
28
170
255
254
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
255
254
253
253
255
255
255
234
181
97
26
34
14
1
2
1

In [16]:
data

((tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.4745, 0.4431, 0.2745],
           [0.0000, 0.0000, 0.0000,  ..., 0.3529, 0.4745, 0.3608],
           [0.0000, 0.0000, 0.0000,  ..., 0.3373, 0.3922, 0.3294],
           ...,
           [0.0667, 0.0706, 0.2078,  ..., 0.4902, 0.5059, 0.0000],
           [0.1961, 0.1333, 0.2039,  ..., 0.3294, 0.4039, 0.1490],
           [0.2078, 0.1647, 0.1333,  ..., 0.0000, 0.0000, 0.1176]],
  
          [[0.0000, 0.0000, 0.0000,  ..., 0.4706, 0.4392, 0.2706],
           [0.0000, 0.0000, 0.0000,  ..., 0.3373, 0.4549, 0.3608],
           [0.0000, 0.0000, 0.0000,  ..., 0.3137, 0.3804, 0.3294],
           ...,
           [0.0902, 0.0980, 0.2549,  ..., 0.4941, 0.5098, 0.0000],
           [0.2431, 0.1843, 0.2471,  ..., 0.3294, 0.4078, 0.1490],
           [0.2588, 0.2078, 0.1765,  ..., 0.0000, 0.0000, 0.1176]],
  
          [[0.0000, 0.0000, 0.0000,  ..., 0.4275, 0.4078, 0.2471],
           [0.0000, 0.0000, 0.0000,  ..., 0.3020, 0.4235, 0.3216],
           [0.0000, 0.

In [10]:
mask_path = '/home/lovis/Documents/dida/images/color_encoded/241.png'
mask = Image.open(mask_path)
mask_array = np.array(mask)
for i in mask_array:
    for x in i:
        if x!=0:
            print(x)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


In [11]:
label_img, nr_labels = ndimage.label(mask_array)

In [14]:
np.unique(mask_array)

array([0, 1, 2, 3], dtype=uint8)